Sudoku

In [37]:
#importação de bibliotecas
from random import randint, choice, uniform
import copy

In [38]:
# Função para criação do tabuleiro seguindo o template pré-definido acima
def create_board()-> list:
    
    matrix = [
        [0, 0, 0, 4],
        [0, 0, 0, 0],
        [2, 0, 0, 3],
        [4, 0, 1, 2]
    ]
    return matrix

In [39]:
# Função para completar os espaços em branco com números aleatórios
def fill_in_spaces(board: list, spaces: list) -> list:
    def is_valid(board: list, num: int, row: int, col: int) -> bool:
        # Verifica se o número já está presente na mesma linha
        for i in range(4):
            if board[row][i] == num:
                return False
        
        # Verifica se o número já está presente na mesma coluna
        for i in range(4):
            if board[i][col] == num:
                return False
        
        # Verifica se o número já está presente no mesmo bloco 2x2
        start_row = (row // 2) * 2
        start_col = (col // 2) * 2
        for i in range(start_row, start_row + 2):
            for j in range(start_col, start_col + 2):
                if board[i][j] == num:
                    return False
        
        return True

    def solve(board: list, spaces: list) -> bool:
        if len(spaces) == 0:
            return True
        
        row, col = spaces[0]
        for num in range(1, 5):
            if is_valid(board, num, row, col):
                board[row][col] = num
                if solve(board, spaces[1:]):
                    return True
                board[row][col] = 0
        
        return False

    solve(board, spaces)
    return board

In [40]:
# Função para exibir o tabuleiro de forma clara para o usuário
def show_board(board: list):
    print(f"|{board[0][0]} {board[0][1]}| |{board[1][0]} {board[1][1]}|")
    print(f"|{board[0][2]} {board[0][3]}| |{board[1][2]} {board[1][3]}|", end='\n\n')
    
    print(f"|{board[2][0]} {board[2][1]}| |{board[3][0]} {board[3][1]}|")
    print(f"|{board[2][2]} {board[2][3]}| |{board[3][2]} {board[3][3]}|", end='\n\n')

In [41]:
# Funções para localizar os espaços em branco
def find_spaces(state: list) -> list:
    spaces = []
    for submatrix in range(len(state)):
        for cell in range (len(state[submatrix])):
            if state[submatrix][cell] == 0:
                spaces.append((submatrix, cell))
    return spaces

In [42]:
# Função para calcular a quantidade de conflitos no tabuleiro
def calculate_conflicts(state: list) -> int:
    conflicts = 0
    
    # Verificando conflitos dentro das submatrizes
    for submatrix in range (len(state)):
        numbers_conflicted_submatrix = []
        for cell in range (len(state[submatrix])):
            number = state[submatrix][cell]
            if number not in numbers_conflicted_submatrix:
                quantity_conflicted_submatrix = state[submatrix].count(number)
                if quantity_conflicted_submatrix > 1:
                    conflicts += quantity_conflicted_submatrix-1
                    numbers_conflicted_submatrix.append(number)
    
    # Verificando conflito na linha
    lines = converct_lines(state)
    for line in range(len(lines)):
        numbers_conflicted_lines = []
        for cell in range (len(lines[line])):
            number = lines[line][cell]
            if number not in numbers_conflicted_lines:
                quantity_conflicted_line = lines[line].count(number)
                if quantity_conflicted_line > 1:
                    conflicts += quantity_conflicted_line-1
                    numbers_conflicted_lines.append(number)
                    
    # Verificando conflitos na coluna
    colunas = converct_colunas(state)
    for coluna in range(len(colunas)):
        numbers_conflicted_colunas = []
        for cell in range(len(colunas[coluna])):
            number = colunas[coluna][cell]
            if number not in numbers_conflicted_colunas:
                quantity_conflicted_coluna = colunas[coluna].count(number)
                if quantity_conflicted_coluna > 1:
                    conflicts += quantity_conflicted_coluna-1
                    numbers_conflicted_colunas.append(number)
            
    return conflicts

In [43]:
# Função para criar as linhas do tabuleiro para verificação de conflito
def converct_lines(state: list) -> list:
    lines = []
    
    lines.append(state[0][0:2] + state[1][0:2])
    lines.append(state[0][2:4] + state[1][2:4])
    lines.append(state[2][0:2] + state[3][0:2])
    lines.append(state[2][2:4] + state[3][2:4])
    
    return lines

In [44]:
def converct_colunas(state: list) -> list:
    colunas = []
    
    colunas.append([state[0][0], state[0][2], state[2][0], state[2][2]])
    colunas.append([state[0][1], state[0][3], state[2][1], state[2][3]])
    colunas.append([state[1][0], state[1][2], state[3][0], state[3][2]])
    colunas.append([state[1][1], state[1][3], state[3][1], state[3][3]])
    
    return colunas

In [45]:
# Função para gerar a população inicial
#def populacao_inicial(populacao_size: int, empty_spaces: list) -> list:
    #populacao = []
    #for individual in range(populacao_size):
        #board = create_board()
        #board = fill_in_spaces(board, empty_spaces)
        #populacao.append(board)
    #return populacao

def populacao_inicial(populacao_size: int, empty_spaces: list) -> list:
    populacao = []
    for individual in range(populacao_size):
        board = create_board()
        filled_board = fill_in_spaces(board, empty_spaces)
        populacao.append(filled_board)
    return populacao

In [46]:
# Função de Cross Over (Cruzamento) de indivíduos
def cross_over(genome_1: list, genome_2: list) -> list:
    g_1 = copy.deepcopy(genome_1)
    g_2 = copy.deepcopy(genome_2)
    cut = randint(0, len(g_1)-1)
    return g_1[:cut] + g_2[cut:], g_2[:cut] + g_1[cut:]

In [47]:
# Função de Mutação que altera uma quantidade específica de elementos da submatriz
def mutation(state: list, quantity_mutations: int, empty_spaces: list) -> list:
    empty_spaces_local = copy.deepcopy(empty_spaces)
    for quantity in range(quantity_mutations):
        space = choice(empty_spaces_local)
        empty_spaces_local.remove(space)
        number = randint(1, 4)
        while number == state[space[0]][space[1]]:
            number = randint(1, 4)
        state[space[0]][space[1]] = number
    return state

In [48]:
# Função fitness para avaliar a população
def fitness(populacao: list) -> list:
    fitness_values = []
    for individual in populacao:
        fitness_values.append(1 - calculate_conflicts(individual)/36)
    return fitness_values

In [49]:
# Função de Roleta Viciada para selecionar indivíduos
def addicted_roulette(populacao: list, fitness: list) -> list:
    fitness_total = sum(fitness)
    fractions = [f/fitness_total for f in fitness]
    random_number = uniform(0,1)
    acum = 0
    for i, individual in enumerate(populacao):
        acum += fractions[i]
        if acum >= random_number:
            return individual
        
    return choice(populacao)

In [50]:
# Função utilizada para selecionar uma determinada quantidade de casais reprodutores
def natural_selection(populacao: list, number_breeding_couples: int) -> list:
    selection = []
    fitness_value = fitness(populacao)
    for individual in range(2*number_breeding_couples):
        selection.append(addicted_roulette(populacao, fitness_value))
    return selection

In [51]:
# Função para verificar se a meta foi alcançada
def meta_test(populacao: list):
    fitness_value = fitness(populacao)
    try:
        position_meta = fitness_value.index(1)
        return populacao[position_meta]
    except (ValueError):
        return -1

In [52]:
# Função para localizar o melhor indivíduo da geração
def best_individual(populacao: list) -> list:
    fitness_value = fitness(populacao)
    position = fitness_value.index(max(fitness_value))
    return (populacao[position], fitness_value[position])

In [53]:
# Função do Algoritmo Genético
def genetic(populacao_size: int, generations: int, empty_spaces: list, quantity_mutations: int, mutation_percentage: int, number_breeding_couples: int):
    
    populacao = populacao_inicial(populacao_size, empty_spaces)
    best = best_individual(populacao)
    print(f" Melhor indivíduo da geração 0 com fitness: {best[1]}")
    
    for generation in range(generations):
        print(f"Geração: {generation}")
        
        meta = meta_test(populacao)
        if meta != -1:
            print(f"\nSolução Encontrada na Geração: {generation}\n")
            return show_board(meta)
        else:
            best_generation = best_individual(populacao)
            if best_generation[1] > best[1]:
                best = best_geration
                print(f'Novo melhor fitness com valor: {best[1]}')
        
        nova_populacao = []
        breeders = natural_selection(populacao, number_breeding_couples)
        males = breeders[:number_breeding_couples]
        females = breeders[number_breeding_couples:]
        for male, female in zip(males, females):
            son_1, son_2 = cross_over(male, female)
            nova_populacao.append(son_1)
            nova_populacao.append(son_2)
            
        meta = meta_test(nova_populacao)
        if meta != -1:
            print(f"\nSolução Encontrada na Geração: {generation}\n")
            return show_board(meta)
        else:
            best_geration = best_individual(nova_populacao)
            if best_geration[1] > best[1]:
                best = best_geration
                print(f"Novo Melhor fitness com valor: {best[1]}")
        
        for individual in nova_populacao:
            if randint(0, 100) < mutation_percentage:
                individual = mutation(individual, quantity_mutations, empty_spaces)
        
        populacao = nova_populacao
        
    print(f"\nALGORITMO ENCERRADO DEVIDO AO NÚMERO DE GERAÇÕES SEM ENCONTRAR UM RESULTADO SATISFATÓRIO.\nO MELHOR INDIVÍDUO OBTEVE O FITNESS DE {best[1]}.\n\n")
    return show_board(best[0])

In [54]:
# Configurações iniciais do algoritmo genético
populacao_size = 1000
generations = 10000
empty_spaces = find_spaces(create_board())

quantity_mutations = 5
if quantity_mutations > len(empty_spaces):
    raise ValueError ("O número de valores alterado por mutação é maior que os espaços vazios!")
    
mutation_percentage = 50
if mutation_percentage > 100:
    raise ValueError ("A porcentagem de chance de mutação é maior que 100%")

number_breeding_couples = 4

# inicialização do algoritmo genético
genetic(populacao_size, generations, empty_spaces, quantity_mutations, mutation_percentage, number_breeding_couples)

 Melhor indivíduo da geração 0 com fitness: 1.0
Geração: 0

Solução Encontrada na Geração: 0

|1 2| |3 4|
|3 4| |2 1|

|2 1| |4 3|
|4 3| |1 2|



### 